# Import Libraries

In [22]:
import numpy as np
import random
from collections import defaultdict
import csv


# Configurations

In [55]:
training_filename = 'train.txt'
test_filename = 'test.txt'
trial_filename = 'trial.txt'

UNKNOWN_WORD="<unk>"
SPACE=" "
GRAPH_START_STATE="pi"
graph_states = ['B-PER','B-LOC','B-ORG', 'B-MISC','I-PER','I-LOC','I-ORG', 'I-MISC','O',GRAPH_START_STATE]


# Tokenize and Process File

In [24]:
def process_file(file_name):
    with open(file_name,'r') as file:
        file_data = file.read();
        file_lines = file_data.splitlines()
                
        sentences = []
        pos_tags = []
        ner_tags = []
        
        for i in range(0,len(file_lines),3):
            sentences.append(file_lines[i].split('\t'))
            pos_tags.append(file_lines[i+1].split('\t'))
            ner_tags.append(file_lines[i+2].split('\t'))
            
        
        return sentences, pos_tags, ner_tags
           

In [25]:
sentences_trial, pos_tags_trial, ner_tags_trial  = process_file(trial_filename)

In [86]:
print(sentences_trial)
print(ner_tags_trial)

[['played', 'on', 'Monday', '(', 'home', 'team', 'in', 'CAPS', ')', ':'], ['American', 'League'], ['Cleveland', '2', 'DETROIT', '1']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-MISC', 'I-MISC'], ['B-ORG', 'O', 'B-ORG', 'O']]


# Create Training and Dev Set

In [27]:
sentences_tr, pos_tags_tr, ner_tags_tr = process_file(training_filename)
sentences_te, pos_tags_te, ner_tags_te = process_file(test_filename)

In [28]:
nTr = len(sentences_tr)
nTr

14000

In [29]:
c=list(zip(sentences_tr, pos_tags_tr, ner_tags_tr))
random.seed(1)
random.shuffle(c)

sentences_tr, pos_tags_tr, ner_tags_tr = zip(*c)

split_point = int(0.8*nTr)

#Validation Set
sentences_val = sentences_tr[split_point:]
pos_tags_val = pos_tags_tr[split_point:]
ner_tags_val = ner_tags_tr[split_point:]


sentences_train = sentences_tr[:split_point]
pos_tags_train = pos_tags_tr[:split_point]
ner_tags_train = ner_tags_tr[:split_point]

# Pre-process data

In [32]:
def flatten_nested_lists(nested_lists):
    result = []
    
    for list_ in nested_lists:
        result+=list_
    
    return result
    

In [43]:
def preProcessIOBTagList(nested_lists):
    result = []
    
    for list_ in nested_lists:
        result.append(GRAPH_START_STATE)
        result+=list_
    
    return result

In [47]:
print(ner_tags_trial)
pp_ner_tags_trial = preProcessIOBTagList(ner_tags_trial)
pp_ner_tags_trial

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-MISC', 'I-MISC'], ['B-ORG', 'O', 'B-ORG', 'O']]


['pi',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'pi',
 'B-MISC',
 'I-MISC',
 'pi',
 'B-ORG',
 'O',
 'B-ORG',
 'O']

# Generic Unsmoothed N-gram Function

In [48]:
##################################
# Generic N-gram function
##################################
def get_N_gram_counts(N, data_list):
    if(N<=0):
        return {};
    
    n_gram_counts = {};
    
    #Form N-gram sequences
    if N>1:
        data_len = len(data_list);
        n_gram_sequence = [];
        
        for i,token in enumerate(data_list):
            if(i>=N-1):
                phrase=""
                for j in reversed(range(N-1)):
                    phrase+=data_list[i-(j+1)];
                    phrase+=SPACE;
                phrase+=token;
                n_gram_sequence.append(phrase);
                
    else:
        n_gram_sequence = data_list;
    
    #Count N-gram sequences
    for phrase in n_gram_sequence:
        if phrase in n_gram_counts.keys():
            n_gram_counts[phrase] = n_gram_counts[phrase] + 1;
        else:
            n_gram_counts[phrase] = 1;
    
    return n_gram_counts;

def get_N_gram_probabilities(N,data_list):
    if(N<=0):
        return {};
    
    n_gram_prob={};
    
    n_gram_counts = get_N_gram_counts(N,data_list);
    if(N==1):
        data_list = [token for token in data_list];
        total_num_words = len(data_list);
        n_gram_prob = { key:n_gram_counts[key]/total_num_words for key in n_gram_counts};
    else:
        n_gram_prefix_counts = get_N_gram_counts(N-1,data_list);
        n_gram_prob = { phrase:(n_gram_counts[phrase]/n_gram_prefix_counts[phrase[:phrase.rfind(" ")]])
                       for phrase in n_gram_counts.keys()}
        
        
    return n_gram_prob;


def get_N_gram_fraction(N,data_list):
    if(N<=0):
        return {};
    
    n_gram_prob={};
    
    n_gram_counts = get_N_gram_counts(N,data_list);
    if(N==1):
        data_list = [token for token in data_list if token != "<s>"];
        total_num_words = len(data_list);
        n_gram_prob = { key:(n_gram_counts[key],total_num_words) for key in n_gram_counts if key!="<s>"};
        
        
    else:
        n_gram_prefix_counts = get_N_gram_counts(N-1,data_list);
        n_gram_prob = { phrase:(n_gram_counts[phrase],n_gram_prefix_counts[phrase[:phrase.rfind(" ")]])
                       for phrase in n_gram_counts.keys()}
        
        
    return n_gram_prob;

In [49]:
get_N_gram_probabilities(2,pp_ner_tags_trial)

{'B-MISC I-MISC': 1.0,
 'B-ORG O': 1.0,
 'I-MISC pi': 1.0,
 'O B-ORG': 0.08333333333333333,
 'O O': 0.75,
 'O pi': 0.08333333333333333,
 'pi B-MISC': 0.3333333333333333,
 'pi B-ORG': 0.3333333333333333,
 'pi O': 0.3333333333333333}

# Kneser Ney Smoothing

In [170]:
def aux_lam_ds_bigram(bigram_keys):
    ds={}
    for key in bigram_keys:
        first_word = key.split()[0]
        if(first_word in ds):
            ds[first_word]+=1
        else:
            ds[first_word]=1
              
    return ds

def aux_p_continuation_ds_bigram(bigram_keys):
    ds={}
    total_bigrams=len(bigram_keys)
    for key in bigram_keys:
        second_word = key.split()[1]
        if(second_word in ds):
            ds[second_word]+=1
        else:
            ds[second_word]=1
    
    prob = {key:ds[key]/total_bigrams for key in ds.keys()}
    
    return prob
    

GENERAL_DISC_FACTOR=0.75
ZERO_DISC_FACTOR=0.5

def get_kn_abs_disc_prob_bigram(data_list):
    modified_prob = {}
    #Do not consider inputs with unknown
    bigram_counts = get_N_gram_counts(2,data_list);
    unigram_counts = get_N_gram_counts(1,data_list);
    
    total_tokens=len(data_list)
    
    discount_factor = GENERAL_DISC_FACTOR;
    
    p_cont = aux_p_continuation_ds_bigram(bigram_counts.keys())
    lam_ds = aux_lam_ds_bigram(bigram_counts.keys())
    
    for phrase in bigram_counts.keys():
        if bigram_counts[phrase] == 1:
            discount_factor=ZERO_DISC_FACTOR=0.5
        else:
            discount_factor=GENERAL_DISC_FACTOR
        first_word=phrase.split()[0];
        second_word=phrase.split()[1];
        bigram_term = max(bigram_counts[phrase] - discount_factor,0)/unigram_counts[first_word]
        lam = (discount_factor*lam_ds[first_word]) /unigram_counts[first_word]
        unigram_term = lam * p_cont[second_word]
        modified_prob[phrase]=bigram_term+unigram_term
    
    
    
    #Handle unknown outside
    modified_prob[UNKNOWN_WORD]=(0.0000027)/total_tokens;
    
    #If any word is unknown replace with unknown
    
    return modified_prob;
    

In [171]:
def aux_lam_ds_bigram_1(bigram_keys):
    ds={}
    for key in bigram_keys:
        first_word = key.split()[0]
        if(first_word in ds):
            ds[first_word]+=1
        else:
            ds[first_word]=1
              
    return ds

def aux_p_continuation_ds_bigram_1(bigram_keys):
    ds={}
    total_bigrams=len(bigram_keys)
    for key in bigram_keys:
        second_word = key.split()[1]
        if(second_word in ds):
            ds[second_word]+=1
        else:
            ds[second_word]=1
    
    prob = {key:ds[key]/total_bigrams for key in ds.keys()}
    
    return prob

def get_kn_abs_disc_prob_bigram_lex(bigram_counts, unigram_counts, vocab_len):
    modified_prob = {}
    #Do not consider inputs with unknown
    #bigram_counts = get_N_gram_counts(2,data_list);
    #unigram_counts = get_N_gram_counts(1,data_list);
    
    #total_tokens=len(data_list)
    
    discount_factor = 0.75;
    
    p_cont = aux_p_continuation_ds_bigram(bigram_counts.keys())
    lam_ds = aux_lam_ds_bigram(bigram_counts.keys())
    
    for phrase in bigram_counts.keys():
        if bigram_counts[phrase] == 1:
            discount_factor=0.5
        else:
            discount_factor=0.75
        first_word=phrase.split()[0];
        second_word=phrase.split()[1];
        bigram_term = max(bigram_counts[phrase] - discount_factor,0)/unigram_counts[first_word]
        lam = (discount_factor*lam_ds[first_word]) /unigram_counts[first_word]
        unigram_term = lam * p_cont[second_word]
        modified_prob[phrase]=bigram_term+unigram_term
    
    #Handle unknown outside
    modified_prob[UNKNOWN_WORD]=(0.0000027)/vocab_len;
    
    #If any word is unknown replace with unknown
    
    return modified_prob;

In [172]:
get_kn_abs_disc_prob_bigram(pp_ner_tags_trial)

{'<unk>': 1.4210526315789474e-07,
 'B-MISC I-MISC': 0.5555555555555556,
 'B-ORG O': 0.75,
 'I-MISC pi': 0.6111111111111112,
 'O B-ORG': 0.06944444444444445,
 'O O': 0.75,
 'O pi': 0.06944444444444445,
 'pi B-MISC': 0.2222222222222222,
 'pi B-ORG': 0.2777777777777778,
 'pi O': 0.3333333333333333}

# Add-k smoothing

SyntaxError: unexpected EOF while parsing (<ipython-input-173-52a34e432596>, line 3)

# Compute Probabilities/DS

# Compute closure of the data structures for faster access

In [174]:
# graph -> Represent as adjacency list { t1: { t2: score}} , also add tag for pi -> unigram prob of first word in sentence


def get_graph_states_prob(bigram_model_prob, graph_states=graph_states):
    result = {}
    for cur_state in graph_states:
        state_transition_prob = {}
        for next_state in graph_states:
            if next_state!=GRAPH_START_STATE:
                tag_sequence=cur_state+SPACE+next_state
                if tag_sequence in bigram_model_prob.keys():
                    state_transition_prob[next_state] = bigram_model_prob[tag_sequence]
                else:
                    state_transition_prob[next_state] = bigram_model_prob[UNKNOWN_WORD]
        result[cur_state] = state_transition_prob
    
    return result
                        

In [175]:
#use pre-processed list so that it includes pi
bigram_model_prob = get_kn_abs_disc_prob_bigram(pp_ner_tags_trial)
print(bigram_model_prob)
get_graph_states_prob(bigram_model_prob,graph_states)

{'pi O': 0.3333333333333333, 'O O': 0.75, 'O pi': 0.06944444444444445, 'pi B-MISC': 0.2222222222222222, 'B-MISC I-MISC': 0.5555555555555556, 'I-MISC pi': 0.6111111111111112, 'pi B-ORG': 0.2777777777777778, 'B-ORG O': 0.75, 'O B-ORG': 0.06944444444444445, '<unk>': 1.4210526315789474e-07}


{'B-LOC': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 1.4210526315789474e-07,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 1.4210526315789474e-07},
 'B-MISC': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 0.5555555555555556,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 1.4210526315789474e-07},
 'B-ORG': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 1.4210526315789474e-07,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 0.75},
 'B-PER': {'B-LOC': 1.4210526315789474e-07,
  'B-MIS

In [176]:
def get_exhaustive_vocab():
    data_list = sentences_tr+sentences_te
    
    vocab=[]
    for word in data_list:
        #vocab
        if word not in vocab:
            vocab.append(word)

In [177]:
# word_tag_dict = { t1: { w1:prob, w2: prob }} 

#flatten lists before calling
def get_word_gen_prob(data_list, ner_tags, graph_states=graph_states):
    result = {}
    #Compute counts for ner_tags
    states_count = {}
    state_word_count = {} # "tag word"
    
    #Initialize all count to 0
    for state in graph_states:
        states_count[state]=0
        result[state]={}
    
    #Count occurence of tags
    for state in ner_tags:
        states_count[state]+=1
    
    
    #print(data_list)
    #print(ner_tags)
    #Count number of occurence of word
    for i,word in enumerate(data_list):
        seq = ner_tags[i]+SPACE+word
        if seq in state_word_count.keys():
            state_word_count[seq]+=1
        else:
            state_word_count[seq]=1
        
    #print(state_word_count)
    #Get smoothed bigram counts
    
    
    
    sm_prob = get_kn_abs_disc_prob_bigram_lex(state_word_count ,states_count, len(set(data_list)))
    
    
    for key in sm_prob.keys():
        if key == UNKNOWN_WORD:
            result[UNKNOWN_WORD]=sm_prob[UNKNOWN_WORD]
        else:
            result[key.split()[0]][key.split()[1]]=sm_prob[key]
    
            
    #Create exhausitive list based on vocabulary
    #for state in graph_states:
    #    for word in vocab:
    #        seq=state+SPACE+word
    #        if seq in sm_prob.keys():
    #            result[state][word]=sm_prob[seq]
    #        else:
    #            result[state][word]=sm_prob[UNKNOWN_WORD]
    
    
    return result
    
            
        
    
    
    
        
        

In [204]:
# word_tag_dict = { t1: { w1:prob, w2: prob }} 

#flatten lists before calling
def get_word_gen_prob(data_list, ner_tags, k=0.9, graph_states=graph_states):
    result = {}
    #Compute counts for ner_tags
    states_count = {}
    state_word_count = {} # "tag word"
    
    #Initialize all count to 0
    for state in graph_states:
        states_count[state]=0
        result[state]={}
    
    #Count occurence of tags
    for state in ner_tags:
        states_count[state]+=1
    
    
    #print(data_list)
    #print(ner_tags)
    #Count number of occurence of word
    for i,word in enumerate(data_list):
        seq = ner_tags[i]+SPACE+word
        if seq in state_word_count.keys():
            state_word_count[seq]+=1
        else:
            state_word_count[seq]=1
        
    #print(state_word_count)
    #Get smoothed bigram counts
    
    #Get vocab count
    V = len(set(data_list))
    
    #sm_prob = get_kn_abs_disc_prob_bigram_lex(state_word_count ,states_count, len(set(data_list)))
    
    for phrase in state_word_count.keys():
        tag=phrase.split()[0]
        word=phrase.split()[1]
        
        result[tag][word]= (state_word_count[word]+k)/(states_count[tag] + k*V)
    
    #to fetch in required format
    for key in sm_prob.keys():
        if key == UNKNOWN_WORD:
            result[UNKNOWN_WORD]=sm_prob[UNKNOWN_WORD]
        else:
            result[key.split()[0]][key.split()[1]]=sm_prob[key]
    
            
    #Create exhausitive list based on vocabulary
    #for state in graph_states:
    #    for word in vocab:
    #        seq=state+SPACE+word
    #        if seq in sm_prob.keys():
    #            result[state][word]=sm_prob[seq]
    #        else:
    #            result[state][word]=sm_prob[UNKNOWN_WORD]
    
    
    return result
    
            

In [178]:
a=[1,2]
b=[34]
a+b

[1, 2, 34]

In [179]:
flattened_sentences = flatten_nested_lists(sentences_trial)
ner_tags_sentences = flatten_nested_lists(ner_tags_trial)
get_word_gen_prob(flattened_sentences,ner_tags_sentences)

{'<unk>': 1.6875e-07,
 'B-LOC': {},
 'B-MISC': {'American': 0.53125},
 'B-ORG': {'Cleveland': 0.28125, 'DETROIT': 0.28125},
 'B-PER': {},
 'I-LOC': {},
 'I-MISC': {'League': 0.53125},
 'I-ORG': {},
 'I-PER': {},
 'O': {'(': 0.07291666666666666,
  ')': 0.07291666666666666,
  '1': 0.07291666666666666,
  '2': 0.07291666666666666,
  ':': 0.07291666666666666,
  'CAPS': 0.07291666666666666,
  'Monday': 0.07291666666666666,
  'home': 0.07291666666666666,
  'in': 0.07291666666666666,
  'on': 0.07291666666666666,
  'played': 0.07291666666666666,
  'team': 0.07291666666666666},
 'pi': {}}

In [372]:
#This data list and ner tags needs to be preprocessed 
def get_next_tag_prob(pp_data_list, pp_ner_tags,k,V, graph_states=graph_states):
    result = {}
    
    #prev_tag_n_cur_word_counts
    bigram_counts= {}
    
    #prev_tag_cur_word_n_tag_counts
    trigram_counts= {}
    
    #Accumulate counts
    for i,word in enumerate(pp_data_list):
        if word!=GRAPH_START_STATE:
            bigram_seq=pp_ner_tags[i-1]+SPACE+word
            trigram_seq=pp_ner_tags[i-1]+SPACE+word+SPACE+pp_ner_tags[i]
            
            if bigram_seq in bigram_counts.keys():
                bigram_counts[bigram_seq]+=1
            else:
                bigram_counts[bigram_seq]=1
                
            if trigram_seq in trigram_counts.keys():
                trigram_counts[trigram_seq]+=1
            else:
                trigram_counts[trigram_seq]=1
    
    #print(trigram_counts)
    #print(bigram_counts)
    #get_kn_abs_disc_prob_bigram_lex(trigram_counts, bigram_counts, len(pp_data_list))
    
    #Cal probability    
    for key in trigram_counts.keys():
        key_list = key.split()
        
        seq_list=[key_list[0],key_list[1]]
        seq_string = key[:key.rfind(' ')]
        #print(seq_string)
        if seq_string not in result.keys():
            result[seq_string]={}
            
        result[seq_string][key_list[2]] = (trigram_counts[key] +k)/(bigram_counts[seq_string]+ k*V)
        result[seq_string]['sm_bigram_count'] = bigram_counts[seq_string]+ k*V
        
    result[UNKNOWN_WORD]=1/V
        
    
    
    return result

In [370]:
pp_sentences_trial=preProcessIOBTagList(sentences_trial)
pp_ner_tags_trial=preProcessIOBTagList(ner_tags_trial)
get_next_tag_prob(pp_sentences_trial,pp_ner_tags_trial,1,10)

{'B-MISC League': {'I-MISC': 0.18181818181818182, 'sm_bigram_count': 11},
 'B-ORG 1': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'B-ORG 2': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O (': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O )': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O :': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O CAPS': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O DETROIT': {'B-ORG': 0.18181818181818182, 'sm_bigram_count': 11},
 'O Monday': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O home': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O in': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O on': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'O team': {'O': 0.18181818181818182, 'sm_bigram_count': 11},
 'pi American': {'B-MISC': 0.18181818181818182, 'sm_bigram_count': 11},
 'pi Cleveland': {'B-ORG': 0.18181818181818182, 'sm_bigram_count': 11},
 'pi played': {'O': 0.18181818181818182,

In [368]:
k=1
V=len(set(flatten_nested_lists(sentences_train)))

pp_sentences_train=preProcessIOBTagList(sentences_train)
pp_ner_tags_train=preProcessIOBTagList(ner_tags_train)
get_next_tag_prob(pp_sentences_train,pp_ner_tags_train,k,V)

{'pi Returns': {'O': 9.202595131827176e-05},
 'O on': {'O': 0.06003719884078031},
 'O Treasuries': {'B-MISC': 9.202595131827176e-05},
 'B-MISC were': {'O': 0.0003680022080132481},
 'O also': {'O': 0.007082152974504249},
 'O in': {'O': 0.10250691777144509},
 'O negative': {'O': 0.0004139834406623735},
 'O territory': {'O': 0.0003680022080132481},
 'O at': {'O': 0.0291292498771389},
 'O minus': {'O': 0.0002760270506509638},
 'O 0.24': {'O': 9.202595131827176e-05},
 'O percent': {'O': 0.011418433263579292},
 'O ,': {'O': 0.1546331595736404},
 'O the': {'O': 0.20599948847235924},
 'O poorest': {'O': 0.00013803257568786233},
 'O result': {'O': 0.0012409228789410792},
 'O after': {'O': 0.015627831128827686},
 'O Canada': {'B-LOC': 0.0017914561322921452},
 'B-LOC and': {'O': 0.008350825956009857},
 'O British': {'B-LOC': 0.0002752798678656634,
  'B-MISC': 0.0024316388328133602,
  'B-ORG': 0.00036703982382088455},
 'B-MISC gilts': {'O': 0.00013803257568786233},
 'O which': {'O': 0.011823018507

In [347]:
a={}
a[str([2,1])]=3
a

{'[2, 1]': 3}

In [366]:
#Compute State Transition Probabilities
#use pre-processed list so that it includes pi

#Training Set
pp_sentences_train=preProcessIOBTagList(sentences_train)
bigram_model_prob_train = get_kn_abs_disc_prob_bigram(pp_sentences_train)
trans_prob_train = get_graph_states_prob(bigram_model_prob_train)

#Entire Training Set
pp_sentences_tr=preProcessIOBTagList(sentences_tr)
bigram_model_prob_tr = get_kn_abs_disc_prob_bigram(pp_sentences_tr)
trans_prob_tr = get_graph_states_prob(bigram_model_prob_tr)

#Compute Lexical Word Gen Prob
#Training Set
flattened_sentences_train = flatten_nested_lists(sentences_train)
ner_tags_sentences_train = flatten_nested_lists(ner_tags_train)
word_gen_prob_train = get_word_gen_prob(flattened_sentences_train,ner_tags_sentences_train)

#Entire Training Set
flattened_sentences_tr = flatten_nested_lists(sentences_tr)
ner_tags_sentences_tr = flatten_nested_lists(ner_tags_tr)
word_gen_prob_tr = get_word_gen_prob(flattened_sentences_tr,ner_tags_sentences_tr)

#Compute next_tag_prob for MEMM

KeyError: 'Returns'

# Viterbi Algorithm - HMM

In [183]:
# Input format
# line =[ , , ]
# graph -> Represent as adjacency list { t1: { t2: score}} , also add tag for pi -> unigram prob of first word in sentence
# word_tag_dict = { t1: { w1:prob, w2: prob }} 
#Unseen and unknown tags can be handled before 
#Unseen words can be handled but unknown words need to be handled here
def viterbi_hmm(line,graph, word_tag_dict):
    T = len(line)
    states = [tag for tag in graph.keys() if tag!=GRAPH_START_STATE] 
    N = len(states)
    viterbi = np.zeros((N,T))
    bp = np.zeros((N,T))
    
    for i in range(N):
        if line[0] in word_tag_dict[states[i]].keys():
            viterbi[i][0] = graph[GRAPH_START_STATE][states[i]] * word_tag_dict[states[i]][line[0]]
        else:
            viterbi[i][0] = graph[GRAPH_START_STATE][states[i]] * word_tag_dict[UNKNOWN_WORD]
        bp[i][0] = 0
    
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range(N):
                if line[t] in word_tag_dict[states[s]].keys():
                    prob_to_state[j] = viterbi[j][t-1] * graph[states[j]][states[s]] * word_tag_dict[states[s]][line[t]]
                else:
                    prob_to_state[j] = viterbi[j][t-1] * graph[states[j]][states[s]] * word_tag_dict[UNKNOWN_WORD]
                        
            viterbi[s][t] = np.max(prob_to_state)
            bp[s][t] = np.argmax(prob_to_state).astype(int)    
    
    path = []
    tag_index = np.argmax(viterbi,axis=0)[T-1].astype(int)
    #print(viterbi)
    #print(bp)
    #print(tag_index)
    
    for i in reversed(range(T)):    
        path.append(states[tag_index])
        tag_index = (bp[tag_index][i]).astype(int) 
        
    path = path[::-1]
    
    return path

# Viterbi algorithm - MEMM

In [373]:
# Input format
# line =[ , , ]
# new_tag_prob  -> { [tprev,word1] : {tcur: } }
def viterbi_memm(line,graph, word_tag_dict):
    T = len(line)
    states = [tag for tag in graph.keys() if tag!=GRAPH_START_STATE] 
    N = len(states)
    viterbi = np.zeros((N,T))
    bp = np.zeros((N,T))
    
    for i in range(N):
        
        viterbi[i][0] = new_tag_prob[[GRAPH_START_STATE,line[0]]][states[i]] 
        bp[i][0] = 0
    
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range(N):
                prob_to_state[j] = viterbi[j][t-1] * new_tag_prob[[states[j],line[t]]][states[s]] 
                        
            viterbi[s][t] = np.max(prob_to_state)
            bp[s][t] = np.argmax(prob_to_state).astype(int)     
    
    path = []
    tag_index = np.argmax(viterbi,axis=0)[T-1].astype(int) 
    
    for i in reversed(range(T)):     
        path.append(states[tag_index])
        tag_index = bp[tag_index][i].astype(int) 
        print(tag_index)
        
    path = path[::-1]
    
    return path

# MEMM 2

In [391]:
# Input format
# line =[ , , ]
# new_tag_prob  -> { 'tprev word1' : {tcur: } }
def viterbi_memm_2(line,graph, new_tag_prob,k):
    T = len(line)
    states = [tag for tag in graph.keys() if tag!=GRAPH_START_STATE] 
    N = len(states)
    viterbi = np.zeros((N,T))
    bp = np.zeros((N,T))
    
    for i in range(N):
        key=GRAPH_START_STATE+SPACE+line[0]
        if key in new_tag_prob.keys():
            if states[i] in new_tag_prob[key].keys():
                viterbi[i][0] = new_tag_prob[key][states[i]]
            else:
                viterbi[i][0] = k/new_tag_prob[key]['sm_bigram_count']
            
        else:
            viterbi[i][0] = new_tag_prob[UNKNOWN_WORD]
        bp[i][0] = 0
    
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range(N):
                key=states[j]+SPACE+line[t]
                if key in new_tag_prob.keys():
                    if states[s] in new_tag_prob[key].keys():
                        prob_to_state[j] = viterbi[j][t-1] * new_tag_prob[key][states[s]] 
                    else:
                        prob_to_state[j] = k / new_tag_prob[key]['sm_bigram_count'] 
                else:
                    prob_to_state[j] = viterbi[j][t-1] * new_tag_prob[UNKNOWN_WORD]
                        
            viterbi[s][t] = np.max(prob_to_state)
            bp[s][t] = np.argmax(prob_to_state).astype(int)     
    
    path = []
    tag_index = np.argmax(viterbi,axis=0)[T-1].astype(int) 
    
    for i in reversed(range(T)):     
        path.append(states[tag_index])
        tag_index = bp[tag_index][i].astype(int) 
        #print(tag_index)
        
    path = path[::-1]
    
    return path

In [388]:
k=0
V=len(set(flatten_nested_lists(sentences_train)))

pp_sentences_train=preProcessIOBTagList(sentences_train)
pp_ner_tags_train=preProcessIOBTagList(ner_tags_train)
res =get_next_tag_prob(pp_sentences_train,pp_ner_tags_train,k,V)
line=sentences_val[0]
corr=ner_tags_val[0]
print(line)
print(corr)
pred=viterbi_memm_2(line,trans_prob_train,res,k)
evaluate_model_span(pred,corr)

['Summaries', 'of', 'French', 'first', 'division']
['O', 'O', 'B-MISC', 'O', 'O']
8
3
8
8
0


(1.0, 1.0, 1.0)

# Evaluation

In [185]:
def evaluate_model_span(predicted_seq, correct_seq):
    T = len(predicted_seq)
    
    num_spans_pred = ['B-' in x for x in predicted_seq].count(True)
    num_spans_ans = ['B-' in x for x in correct_seq].count(True)
    
    correct_pred=0
    
    for i in range(len(correct_seq)):
        
        if 'B-' in correct_seq[i]:
            if correct_seq[i]==predicted_seq[i]:
                flag=1;
                tag_type = correct_seq[i][2:]
                j=i+1
                while( j<T and (correct_seq[j] == 'I-'+tag_type or predicted_seq[j] == 'I-'+tag_type)):
                    #print("here")
                    if (correct_seq[j]!=predicted_seq[j]):
                        flag=0;
                    
                    j=j+1             
                
                if(flag==1):
                    correct_pred+=1;
                               
                i=j-1
                
                
    precision = correct_pred/num_spans_pred;
    recall = correct_pred/num_spans_ans;
    
    f_score = (2*precision*recall)/(precision+recall)
                
    return precision, recall, f_score     


In [186]:
a=[1,2,3]

In [187]:
a=['B-ORG', 'O', 'B-ORG', 'O','B-MISC', 'I-MISC']
['B-' in x for x in a].count(True)

3

In [188]:
corr=['O','B-PER','I-PER','B-LOC','O','O','B-MISC','B-MISC', 'I-MISC','I-MISC']
pred=['B-PER','B-PER','I-PER','B-LOC','O','O','O','B-LOC', 'I-MISC','I-LOC']

In [189]:
flatten_nested_lists([corr,pred])

['O',
 'B-PER',
 'I-PER',
 'B-LOC',
 'O',
 'O',
 'B-MISC',
 'B-MISC',
 'I-MISC',
 'I-MISC',
 'B-PER',
 'B-PER',
 'I-PER',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-MISC',
 'I-LOC']

In [190]:
evaluate_model_span(pred,corr)

(0.5, 0.5, 0.5)

In [191]:
a[0][2:]

'ORG'

# Analysis - HMM

In [196]:
#Validation Set
#Use train set here instead of tr set 
pred=[]
for line in sentences_val:
    ans = viterbi_hmm(line,trans_prob_train, word_gen_prob_train)
    pred.append(ans)

In [197]:
pred

[['O', 'I-ORG', 'B-MISC', 'O', 'I-MISC'],
 ['O', 'B-PER', 'B-PER', 'O', 'B-LOC', 'O', 'B-PER'],
 ['O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'B-PER',
  'B-PER',
  'B-PER',
  'O'],
 ['B-PER', 'B-PER', 'O', 'B-PER', 'I-LOC', 'B-PER', 'O', 'B-PER'],
 ['B-LOC', 'B-PER', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'B-PER', 'B-PER', 'B-PER', 'B-PER'],
 ['B-PER', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'B-PER'],
 ['O', 'O', 'B-LOC', 'B-PER', 'B-PER', 'B-PER', 'O', 'O', 'O', 'B-PER', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER',
  'B-PER'],
 ['O', 'B-PER', 'I-PER', 'O', 'B-LOC',

In [198]:
ner_tags_trial

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B-MISC', 'I-MISC'],
 ['B-ORG', 'O', 'B-ORG', 'O']]

In [199]:
evaluate_model_span(flatten_nested_lists(pred),flatten_nested_lists(ner_tags_val))

(0.20427750906892383, 0.5923734385272846, 0.3037932003371734)

# Analysis - MEMM

In [395]:
k=0
V=len(set(flatten_nested_lists(sentences_train)))
pp_sentences_train=preProcessIOBTagList(sentences_train)
pp_ner_tags_train=preProcessIOBTagList(ner_tags_train)

res =get_next_tag_prob(pp_sentences_train,pp_ner_tags_train,k,V)
pred=[]
for line in sentences_val:
    ans = viterbi_memm_2(line,trans_prob_train,res, k)
    pred.append(ans)
evaluate_model_span(flatten_nested_lists(pred),flatten_nested_lists(ner_tags_val))

(0.5048686928297433, 0.749945211483673, 0.6034741204479324)

# Word Embeddings

In [200]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [201]:
glove_input_file = 'glove.840B.300d.txt'
word2vec_output_file = 'glove.840B.300d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)




In [212]:
#Read and vectorize Glove 

def loadGloveModel(gloveFile):
    print("Loading File!")
    f = open(gloveFile,'r')
    model = {}
    i=0;
    for line in f:
        splitLine = line.replace("\n","").split(" ")
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word.lower()] = embedding
    print("File Loaded!")
    return model

gloveFile='glove.840B.300d.txt'

glove_model_vectors = loadGloveModel(gloveFile)

Loading File!
File Loaded!


In [413]:
len(glove_model_vectors.keys())

1702926

In [284]:
#print(training_data_obama_we)

def vectorize_tokens(data_list):
    #300 is dim for glove
    vec=np.zeros(300)
    
    glove_keys = glove_model_vectors.keys()
    
    for token in data_list:
        if token in glove_keys:
            vec+=glove_model_vectors[token]
            
    vec=vec/len(data_list)
    return vec

In [285]:
vectorize_tokens(['american'])

array([ 0.2539   , -0.36895  ,  0.26033  , -0.10035  , -0.58879  ,
        0.44728  ,  0.052015 ,  0.78105  , -0.66301  , -1.1887   ,
       -0.066125 ,  0.86127  , -0.39702  , -0.18537  ,  0.43213  ,
       -0.14023  , -0.046757 , -1.6801   ,  0.34622  ,  0.79814  ,
        0.15617  , -0.3673   , -0.22443  ,  0.19137  , -0.28632  ,
       -0.5484   , -0.077039 , -0.24923  ,  0.36135  ,  0.61197  ,
       -0.44122  ,  0.16953  ,  0.63045  , -0.084609 , -0.21828  ,
        0.25694  , -0.23095  , -0.094954 ,  0.16437  ,  0.54833  ,
        0.5678   ,  0.74866  ,  0.088466 ,  0.38904  ,  0.14308  ,
       -0.34348  ,  0.55417  , -0.38818  ,  0.081317 , -0.015508 ,
       -0.65818  , -0.3079   , -0.2125   ,  0.16286  , -0.42047  ,
        0.013507 ,  0.017612 , -0.15186  , -0.23968  , -0.36223  ,
        0.032989 , -0.15466  , -0.58381  , -0.15278  ,  0.1111   ,
       -0.4149   ,  0.33809  , -0.0098793,  0.45988  , -0.44008  ,
        0.083382 , -0.33664  , -0.2418   ,  0.42795  ,  0.0832

In [266]:
def is_Number(string):
    try:
        float(string)
        return 1
    except:
        return 0

In [404]:
#trial
pos_tags_univ = list(set(flatten_nested_lists(pos_tags_tr)+flatten_nested_lists(pos_tags_te)))
len(pos_tags_univ)
print(pos_tags_univ)

['VB', 'JJ', ')', 'LS', 'FW', "''", 'WP$', 'NN|SYM', 'WDT', 'NNPS', 'EX', 'CC', 'TO', 'VBZ', ',', 'RB', 'UH', 'WRB', 'VBD', 'MD', 'PDT', 'WP', 'PRP$', 'POS', 'VBN', '.', 'NNP', 'CD', 'JJR', '$', 'VBG', 'RBR', 'DT', 'JJS', 'RBS', '(', 'NNS', ':', 'NN', 'IN', 'SYM', '"', 'PRP', 'RP', 'VBP']


In [411]:
def feature_extractor(flt_sentences, pos_tags_data):
    N=len(flt_sentences)
    d=303
    feat=np.zeros((N,d))
    label=np.zeros(N)
    words=[]
    nouns=['NN','NNS','NNP','NNPS']
    #print(pos_tags_univ)
    glove_keys = glove_model_vectors.keys()
    for i,word in enumerate(flt_sentences):
        #Probability as continuous feature -- add later
        #print(word)
        #isCapitalized
        feat[i][0]=int(word[0].isupper())
        #isNumber
        feat[i][1]=is_Number(word)
        #is Noun
        if pos_tags_data[i] in nouns:
            feat[i][2]=1
        else:
            feat[i][2]=0
        #POS Tag

        #Word Embedding
        #fetch for lower case
        word_lower = word.lower()
        if word_lower in glove_keys:
            feat[i][3:303]=glove_model_vectors[word_lower]
        else:
            feat[i][3:303]=np.zeros(300)
            
        label[i]=pos_tags_univ.index(pos_tags_data[i])
        words.append(word)
        
    return feat,label,words
        

sample=['I','love','California','1']
pos_tags_data=['JJ','VB','NN','NNP']
feature_extractor(sample,pos_tags_data)

(array([[ 1.      ,  0.      ,  0.      , ...,  0.16495 ,  0.18757 ,
          0.53874 ],
        [ 0.      ,  0.      ,  0.      , ...,  0.39089 , -0.16994 ,
          0.092135],
        [ 1.      ,  0.      ,  1.      , ..., -0.049514, -0.25006 ,
          0.40112 ],
        [ 0.      ,  1.      ,  1.      , ..., -0.61465 ,  0.1773  ,
          0.1659  ]]),
 array([ 1.,  0., 38., 26.]),
 ['I', 'love', 'California', '1'])

In [312]:
flt_sen = flatten_nested_lists(sentences_train)
flt_pos = flatten_nested_lists(pos_tags_train)
fTr =  feature_extractor(flt_sen,flt_pos)

In [315]:
print(flt_sen[0])
print(flt_pos[0])
print(len(fTr[0][:]))

Returns
NNS
303


{'pi O': 0.3333333333333333, 'O O': 0.75, 'O pi': 0.06944444444444445, 'pi B-MISC': 0.2222222222222222, 'B-MISC I-MISC': 0.5555555555555556, 'I-MISC pi': 0.6111111111111112, 'pi B-ORG': 0.2777777777777778, 'B-ORG O': 0.75, 'O B-ORG': 0.06944444444444445, '<unk>': 1.4210526315789474e-07}


{'B-LOC': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 1.4210526315789474e-07,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 1.4210526315789474e-07},
 'B-MISC': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 0.5555555555555556,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 1.4210526315789474e-07},
 'B-ORG': {'B-LOC': 1.4210526315789474e-07,
  'B-MISC': 1.4210526315789474e-07,
  'B-ORG': 1.4210526315789474e-07,
  'B-PER': 1.4210526315789474e-07,
  'I-LOC': 1.4210526315789474e-07,
  'I-MISC': 1.4210526315789474e-07,
  'I-ORG': 1.4210526315789474e-07,
  'I-PER': 1.4210526315789474e-07,
  'O': 0.75},
 'B-PER': {'B-LOC': 1.4210526315789474e-07,
  'B-MIS

In [268]:
int(is_Number('1'))

1

In [ ]:
#Important 

In [412]:
flt_sentences = flatten_nested_lists(sentences_train)
flt_pos_tags = flatten_nested_lists(pos_tags_train)
flt_ner_tags = flatten_nested_lists(ner_tags_train)

feat,label,words = feature_extractor(flt_sentences,flt_pos_tags)

In [410]:
k=0
V=len(set(flatten_nested_lists(sentences_train)))
pp_sentences_train=preProcessIOBTagList(sentences_train)
pp_ner_tags_train=preProcessIOBTagList(ner_tags_train)

res =get_next_tag_prob(pp_sentences_train,pp_ner_tags_train,k,V)


In [409]:

def add_feature_templates():
    
    #cur_tag/prev_tag,cur_word
    
    


array([ 1.0000e+00,  0.0000e+00,  0.0000e+00,  5.7486e-01,  7.0326e-02,
       -2.5808e-01,  3.2482e-01, -1.7873e-01, -7.2740e-01, -3.2019e-01,
        7.3981e-02, -3.2846e-01, -8.5944e-01, -5.7321e-01,  7.6129e-01,
       -3.3751e-02, -1.0564e-02,  1.0546e+00, -5.1391e-01, -4.2129e-01,
       -2.6120e-01,  2.7952e-01,  7.8078e-02, -2.7215e-01,  1.2905e-01,
        1.7239e-01, -4.9920e-03,  2.9988e-01, -3.6435e-01, -3.0291e-01,
        4.6381e-01,  2.8826e-01,  4.4868e-02, -7.2278e-02,  2.6276e-01,
        4.5824e-01,  1.3751e-01,  3.4337e-01, -1.8124e-02,  9.7508e-02,
       -5.5806e-01, -2.0159e-01,  7.8213e-01,  4.1889e-01,  2.7571e-01,
        6.6235e-01, -1.8328e-01,  6.1244e-01,  1.2024e-01,  1.1511e+00,
       -6.2435e-02,  5.7438e-01,  9.3206e-01,  2.6103e-01, -9.1443e-03,
       -3.0391e-01,  1.1360e-01,  1.3088e-02,  2.6707e-01,  1.7599e-01,
       -8.3153e-01,  2.9613e-01,  1.7082e-01, -2.4875e-01, -2.7213e-02,
        3.9247e-01,  6.3329e-01, -8.2152e-02,  6.0572e-01,  4.71

In [396]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X, y)

NameError: name 'X' is not defined

# Again

In [220]:
def get_unique(lst):
    result = []
    for item in lst:
        if item not in result:
            result+=item
    return result

In [232]:
from nltk.util import ngrams
import nltk

In [236]:
flt_ner_tags_train=flatten_nested_lists(ner_tags_train)

In [245]:
#Kn for trigram
train_tags_triigrm = list(nltk.trigrams(flt_ner_tags_train))
freq_dist = nltk.FreqDist(train_tags_triigrm)
kneser_ney = nltk.KneserNeyProbDist(freq_dist)
print(len(set(train_tags_bigrm)))

52


0.0

# Generate test output 

In [183]:
#result = { PER : [] }
def gen_tag_indexes(pred):
    result = defaultdict(list)
    T = len(pred)
    for i in range(len(pred)):
        
        if 'B-' in pred[i]:
            flag=1;
            tag_type = pred[i][2:]
            j=i+1
            while j<T and (pred[j] == 'I-'+tag_type) :
                j=j+1             
                
            j=j-1 #To get to last token
            result[tag_type].append(str(i)+'-'+str(j))
            #if i==j:
            #    result[tag_type].append(str(i))       
            #else:
            #    result[tag_type].append(str(i)+'-'+str(j))
    return result

In [1]:
word_test

NameError: name 'word_test' is not defined

In [186]:
pred=['B-PER','B-PER','I-PER','B-LOC','O','O','O','B-MISC', 'I-MISC','I-LOC']

In [191]:
a= gen_tag_indexes(pred)

In [195]:
' '.join(a['PER'])

'0 1-2'

# Create Excel for Kaggle

In [201]:
def write_result(filename,preds):
    with open(filename, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Type','Prediction'])
        
        filewriter.writerow(['ORG',' '.join(preds['ORG'])])
        filewriter.writerow(['MISC',' '.join(preds['MISC'])])
        filewriter.writerow(['PER',' '.join(preds['PER'])])
        filewriter.writerow(['LOC',' '.join(preds['LOC'])])
        

In [203]:
write_result('temp.csv', a)